**현재 세션(1개)의 앞 부분의 10개 클릭 **를 대상으로 MLP, Gaussian Naive Bayes, Decision Tree, XGBoost, Logistic Regression, Linear SVM을 사용해서 구매 예측

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
# 불러오기
import pandas as pd

온라인 = pd.read_csv('온라인_전처리_final_32columns.csv')

# 각 clnt_id별 session이 바뀌는 지점 index 저장
idx1 = 온라인.unique_id.drop_duplicates().index.tolist()
idx2 = idx1[1:] + [len(온라인)]

In [3]:
온라인.head()

,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


In [4]:
온라인.columns[3:-2]

Index(['hit_seq', 'hit_pss_tm', 'time_length', 'holiday_diff', 'keyword',
       'action_type_0', 'action_type_1', 'action_type_2', 'action_type_3',
       'action_type_4', 'biz_unit_A01', 'biz_unit_A02', 'biz_unit_A03',
       'sess_dt_201907', 'sess_dt_201908', 'sess_dt_201909', 'trfc_src_DIRECT',
       'trfc_src_PORTAL_1', 'trfc_src_PORTAL_2', 'trfc_src_PORTAL_3',
       'trfc_src_PUSH', 'trfc_src_WEBSITE', 'trfc_src_unknown',
       'dvc_ctg_nm_PC', 'dvc_ctg_nm_mobile_app', 'dvc_ctg_nm_mobile_web',
       'dvc_ctg_nm_unknown'],
      dtype='object')

In [5]:
len(온라인.columns[3:-2])

27

# <br>
# 종속변수 생성
다음 세션의 구매 여부

In [7]:
# 다음 세션의 구매를 예측하기 위한 종속변수 생성
구매여부 = 온라인[['clnt_id', 'sess_id', 'buy']].groupby(['clnt_id', 'sess_id']).sum()
구매여부.buy = 구매여부.buy.apply(lambda x:0 if x == 0 else 1)
구매여부 = 구매여부.sort_index()
구매여부 = 구매여부.reset_index()
# 각 clnt_id별로 shift로 1행씩 올림
구매여부.buy = 구매여부.buy.shift(-1)
g = 구매여부.groupby('clnt_id')
# 1행씩 올리면 각 clnt_id별로 마지막 값은 다음 id의 값, 따라서 마지막 행 제거
구매여부.drop(g.tail(1).index, axis=0, inplace = True)

In [8]:
구매여부['unique_id'] = list(map(lambda x,y: str(x)+'_'+str(y), 구매여부.clnt_id, 구매여부.sess_id))

In [9]:
구매여부

,clnt_id,sess_id,buy,unique_id
0,1,1,0.0,1_1
2,2,1,0.0,2_1
3,2,2,0.0,2_2
4,2,3,0.0,2_3
5,2,4,0.0,2_4
...,...,...,...,...
367141,72427,3,0.0,72427_3
367143,72428,1,0.0,72428_1
367144,72428,2,0.0,72428_2
367145,72428,3,0.0,72428_3


# <br>
# 클릭 10개  로그 대상
클릭 10개 미만 세션들 삭제, 10개 이상 클릭 로그들 삭제

In [33]:
# 클릭 10개 미만 세션들은 삭제
temp = pd.DataFrame(온라인.groupby('unique_id')['hit_seq'].count())
클릭10개미만 = temp[temp.hit_seq < 10].index
클릭10개미만

Index(['10000_1', '10000_2', '10001_1', '10003_1', '10004_1', '10004_2',
       '10005_1', '10005_2', '10005_3', '10005_4',
       ...
       '9_24', '9_28', '9_29', '9_3', '9_30', '9_31', '9_4', '9_7', '9_8',
       '9_9'],
      dtype='object', name='unique_id', length=280908)

In [34]:
온라인2 = 온라인[~온라인.unique_id.isin(클릭10개미만)].copy()

In [35]:
# 10개 그 뒤의 hit_seq 삭제
온라인2 = 온라인2[온라인2.hit_seq < 11].copy()

In [36]:
# 10개까지의 hit_seq만 들어갔는지 확인
온라인2.hit_seq.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [37]:
온라인2.drop(['clnt_id', 'sess_id', 'trans_id', 'buy'], axis=1, inplace=True)

In [38]:
def to_flat(df):
    cc = df.groupby(['unique_id']).cumcount() + 1
    flat_df = df.set_index(['unique_id', cc]).unstack().sort_index(1, level=1)
    flat_df.columns = ['_'.join(map(str,i)) for i in flat_df.columns]
    flat_df.reset_index(inplace=True)
    return flat_df

In [39]:
온라인2 = to_flat(온라인2)
온라인2 = 온라인2.merge(구매여부, left_on='unique_id', right_on='unique_id')
온라인2.sort_values(by=['clnt_id','sess_id'], inplace=True)
온라인2.reset_index(drop=True, inplace=True)

In [40]:
온라인2.head()

,unique_id,action_type_0_1,action_type_1_1,action_type_2_1,action_type_3_1,action_type_4_1,biz_unit_A01_1,biz_unit_A02_1,biz_unit_A03_1,dvc_ctg_nm_PC_1,...,trfc_src_DIRECT_10,trfc_src_PORTAL_1_10,trfc_src_PORTAL_2_10,trfc_src_PORTAL_3_10,trfc_src_PUSH_10,trfc_src_WEBSITE_10,trfc_src_unknown_10,clnt_id,sess_id,buy
0,2_1,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,2,1,0.0
1,2_8,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,2,8,0.0
2,2_10,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,2,10,0.0
3,2_15,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,2,15,0.0
4,2_16,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,2,16,0.0


In [46]:
# 10일동안 * feature 27 + clnt_id&sess_id&unique_id&buy = 274 columns
온라인2.shape

(75233, 274)

# <br>
# [1-3]현재 세션(1개)의 앞 부분의 10개 클릭 로그 사용


In [47]:
from keras.preprocessing import sequence
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0

In [50]:
features = 온라인2.columns[1:-3]

In [51]:
온라인_x = 온라인2.iloc[:, 1:-3]
온라인_x = np.array(온라인_x)

In [52]:
# session 당 구매 여부
온라인_y = 온라인2.buy

In [53]:
# 기존 function에서 필요한 부분만 가져옴
def make_padding_and_oversample(X, Y, length=10):
    smote = SMOTE(random_state=0)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)
    return X_resampled, Y_resampled

In [54]:
X_resampled, Y_resampled = make_padding_and_oversample(온라인_x, 온라인_y, length=10)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, Y_resampled, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

(86375, 270)
(37019, 270)


# <br>
# 모델링

In [56]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import xgboost as xgb

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.layers import Dropout

import pickle
from joblib import dump, load

In [57]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [58]:
scores = dict()

### Gaussian Naive Bayes

In [59]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.57
F1 score: 0.65
Precision: 0.54
Recall: 0.81


In [60]:
scores['Gaussian Naive Bayes'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                                  precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [61]:
# dump(clf, '1-3-Gaussian.joblib')

### Decision Tree

In [62]:
clf2 = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.81
F1 score: 0.81
Precision: 0.79
Recall: 0.82


In [63]:
scores['Decision Tree'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                           precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [64]:
feature_check = {}
for name, importance in zip(features, clf2.feature_importances_):
    feature_check[name] = importance

In [65]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
trfc_src_DIRECT_9,0.149946
action_type_3_2,0.102040
trfc_src_DIRECT_4,0.081251
trfc_src_PUSH_3,0.042576
sess_dt_201908_10,0.034527
...,...
hit_seq_7,0.000000
biz_unit_A01_3,0.000000
biz_unit_A03_7,0.000000
biz_unit_A02_7,0.000000


In [66]:
# dump(clf2, '1-3-DecisionTree.joblib')

### XGBOOST

In [67]:
clf3 = xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=3, verbosity=2, random_state=0).fit(X_train, y_train)
y_pred = clf3.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

[15:13:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[15:13:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[15:13:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[15:13:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[15:13:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15:13:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15:

In [68]:
scores['XGboost'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [69]:
feature_check = {}
for name, importance in zip(features, clf3.feature_importances_):
    feature_check[name] = importance

In [70]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
sess_dt_201908_2,0.059765
action_type_3_4,0.048704
sess_dt_201907_1,0.045531
dvc_ctg_nm_unknown_2,0.044836
action_type_3_3,0.044706
...,...
trfc_src_PORTAL_2_5,0.000000
trfc_src_PORTAL_3_5,0.000000
trfc_src_PUSH_5,0.000000
trfc_src_WEBSITE_5,0.000000


In [71]:
# dump(clf3, '1-3-xgb.joblib')

### Logistic Regression

In [72]:
clf4 = LogisticRegression(max_iter=1000, random_state=0).fit(X_train, y_train)
y_pred = clf4.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.58
F1 score: 0.60
Precision: 0.57
Recall: 0.63


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [73]:
scores['Logistic Regression'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [74]:
# dump(clf4, '1-3-LogisticRegression.joblib')

### Linear SVM

In [75]:
clf5 = LinearSVC(random_state=0).fit(X_train, y_train)
y_pred = clf5.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.51
F1 score: 0.66
Precision: 0.50
Recall: 0.98


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [76]:
scores['Linear SVM'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [77]:
# dump(clf4, '1-3-LinearSVM.joblib')

### MLP(DNN)

In [78]:
def models(train):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(train.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr= 0.001, rho = 0.9), metrics=['acc', f1_m, precision_m, recall_m])
    return model

In [79]:
model = models(X_train)
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                8672      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 9,217
Trainable params: 9,217
Non-trainable params: 0
_________________________________________________________________


In [80]:
history = model.fit(X_train, y_train, epochs=25, batch_size=1000, validation_data=(X_test, y_test), verbose=2, shuffle=True)


Train on 86375 samples, validate on 37019 samples
Epoch 1/25
 - 1s - loss: 1.6380 - acc: 0.5063 - f1_m: 0.4436 - precision_m: 0.5024 - recall_m: 0.5201 - val_loss: 1.4200 - val_acc: 0.5161 - val_f1_m: 0.6509 - val_precision_m: 0.5083 - val_recall_m: 0.9057
Epoch 2/25
 - 1s - loss: 1.2571 - acc: 0.5096 - f1_m: 0.4204 - precision_m: 0.5038 - recall_m: 0.5152 - val_loss: 1.1796 - val_acc: 0.5032 - val_f1_m: 0.1678 - val_precision_m: 0.4976 - val_recall_m: 0.1012
Epoch 3/25
 - 1s - loss: 1.1512 - acc: 0.5113 - f1_m: 0.4141 - precision_m: 0.5175 - recall_m: 0.5103 - val_loss: 1.2136 - val_acc: 0.5200 - val_f1_m: 0.6556 - val_precision_m: 0.5097 - val_recall_m: 0.9193
Epoch 4/25
 - 1s - loss: 1.0826 - acc: 0.5136 - f1_m: 0.4232 - precision_m: 0.5080 - recall_m: 0.5245 - val_loss: 1.2340 - val_acc: 0.5063 - val_f1_m: 0.1079 - val_precision_m: 0.5354 - val_recall_m: 0.0601
Epoch 5/25
 - 1s - loss: 1.0413 - acc: 0.5162 - f1_m: 0.4242 - precision_m: 0.5217 - recall_m: 0.5174 - val_loss: 0.9045 

In [81]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)

37019/37019 [==============================] - 2s 41us/step


In [82]:
scores['DNN'] = [accuracy, f1_score, precision, recall]

In [83]:
print(loss)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

0.7110973841647648
0.5462599992752075
0.3664796054363251
0.5919910669326782
0.27579590678215027


In [84]:
# model.save('1-3-DNN.h5')

In [85]:
pd.DataFrame.from_dict(scores, orient='index', columns=['Accuracy', 'F1-Score', 'Precision', 'Recall'])

,Accuracy,F1-Score,Precision,Recall
Gaussian Naive Bayes,0.566898,0.651115,0.542419,0.814293
Decision Tree,0.805262,0.807550,0.792466,0.823219
XGboost,0.889543,0.874789,1.000000,0.777445
Logistic Regression,0.578811,0.597750,0.568199,0.630545
Linear SVM,0.506416,0.662280,0.501413,0.975127
DNN,0.546260,0.366480,0.591991,0.275796
